In [1]:
using DataFrames, CSV, Pipe
ENV["COLUMNS"] =200
ENV["LINES"] = 10

10

In [2]:
df = CSV.File("genome_list.tsv", delim = "\t") |> DataFrame

┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 2. Filling remaining columns with `missing`
└ @ CSV /Users/danielgomezperez/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 3. Filling remaining columns with `missing`
└ @ CSV /Users/danielgomezperez/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 4. Filling remaining columns with `missing`
└ @ CSV /Users/danielgomezperez/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 5. Filling remaining columns with `missing`
└ @ CSV /Users/danielgomezperez/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 12. Filling remaining columns with `missing`
└ @ CSV /Users/danielgomezperez/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 

,species_name,proteomes,lifestyle,genomes,accession,alternative_name
,String,Float64,String,Int64,String?,String?
1,Alternaria_alternata,3.0,N,1,"GCA_001572055.1,GCA_001642055.1,GCA_004154755.1",missing
2,Alternaria_arborescens,3.0,N,1,"GCA_004154815.1,GCA_004154835.1,GCA_004155955.1",missing
3,Alternaria_gaisen,1.0,N,1,GCA_004156025.2,missing
4,Alternaria_tenuissima,7.0,N,1,"GCA_004154735.1,GCA_004154745.1,GCA_004154765.1,GCA_004154845.1,GCA_004156015.1,GCA_004156035.1,GCA_004168565.1",missing
5,Ascochyta_rabiei,1.0,N,1,GCA_001630375.1,missing
6,Bipolaris_zeicola,1.0,N,1,GCF_000523435.1,Cochliobolus_carbonum
7,Bipolaris_maydis,2.0,N,1,"GCA_000338975.1,GCA_000354255.1",Cochliobolus_heterostrophus
8,Bipolaris_oryzae,1.0,N,1,GCA_000523455.1,Cochliobolus_miyabeanus
9,Bipolaris_sorokiniana,2.0,H,1,"GCA_000338995.1,GCA_013416765.1",Cochliobolus_sativus


In [3]:
df = @pipe df |> 
    dropmissing(_,:accession)|>
    select(_,:accession =>( x-> split.(x,","))=>:accession,:) |>
    flatten(_,:accession)

,accession,species_name,proteomes,lifestyle,genomes,alternative_name
,SubStri…,String,Float64,String,Int64,String?
1,GCA_001572055.1,Alternaria_alternata,3.0,N,1,missing
2,GCA_001642055.1,Alternaria_alternata,3.0,N,1,missing
3,GCA_004154755.1,Alternaria_alternata,3.0,N,1,missing
4,GCA_004154815.1,Alternaria_arborescens,3.0,N,1,missing
5,GCA_004154835.1,Alternaria_arborescens,3.0,N,1,missing
6,GCA_004155955.1,Alternaria_arborescens,3.0,N,1,missing
7,GCA_004156025.2,Alternaria_gaisen,1.0,N,1,missing
8,GCA_004154735.1,Alternaria_tenuissima,7.0,N,1,missing
9,GCA_004154745.1,Alternaria_tenuissima,7.0,N,1,missing


## Read dbcan file 

In [4]:
using Glob

In [5]:
dfs = []
for i in Glob.glob("cazy/*")
    d = CSV.File(i,delim="\t") |> DataFrame
    d = @pipe d |>
    filter(r -> r."#ofTools" >2,_) |>
    groupby(_,:DIAMOND) |>
    combine(_, nrow)
    accession = split(i,"/")[2]
    accession = SubString(accession, 1, length(accession)- 4)
    rename!(d,["cazy",accession])
    push!(dfs,d)
end

In [6]:
X = coalesce.(outerjoin(dfs...,on="cazy"),0)

,cazy,Albugo_laibachii.ENA1_B,GCA_000002495.2,GCA_000002525.1,GCA_000002655.1,GCA_000002855.2,GCA_000002945.2,GCA_000006445.2,GCA_000011425.1,GCA_000027005.1,GCA_000091065.2,GCA_000142945.1
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,GT2,2,8,8,5,9,0,5,5,4,0,2
2,GH5_12,1,1,2,0,1,0,1,1,1,0,3
3,GH37,5,1,1,0,0,0,1,0,1,0,3
4,GH3,2,14,5,10,13,0,3,12,1,0,8
5,GT48,8,1,2,0,1,0,2,0,3,0,6
6,GH31,1,6,1,5,5,1,3,4,1,0,4
7,GT20,6,2,3,7,4,3,3,3,3,0,7
8,GT4,3,3,3,3,4,1,3,3,3,0,1
9,GT66,4,1,1,1,1,0,1,1,1,0,2


In [11]:
sort!(X,:cazy)

,cazy,Albugo_laibachii.ENA1_B,GCA_000002495.2,GCA_000002525.1,GCA_000002655.1,GCA_000002855.2,GCA_000002945.2,GCA_000006445.2,GCA_000011425.1,GCA_000027005.1,GCA_000091065.2,GCA_000142945.1
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,AA0,0,1,0,0,0,0,0,0,0,0,0
2,AA1,1,1,1,1,0,0,1,0,0,0,0
3,AA1+CBM1,0,0,0,0,0,0,0,0,0,0,0
4,AA10,0,0,0,0,0,0,0,0,0,0,0
5,AA11,0,5,0,1,3,0,0,2,0,0,0
6,AA11+1.14.99.53,0,0,0,1,0,0,0,0,0,0,0
7,AA11+CBM18,0,0,0,0,0,0,0,0,0,0,0
8,AA12,0,4,0,0,0,0,0,0,0,0,0
9,AA12+AA8,0,0,0,0,0,0,0,0,0,0,0


In [21]:
lj = leftjoin(DataFrame([[names(X)]; collect.(eachrow(X))], 
[:column; Symbol.(axes(X, 1))]),select(df,:accession,:lifestyle),on="column" => "accession")

,column,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,GCA_000002495.2,1,1,0,0,5,0,0,4,0,0,1,0,0,0,0,1,0,0,2,6,0,0,0,0,2,0,0,0,3,3,1,0,1,0,10,0,0
2,GCA_000002525.1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,GCA_000002655.1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,8,0,0
4,GCA_000002855.2,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,3,6,0,1,0,0,0,0,0,0,2,1,1,0,0,0,24,0,0
5,GCA_000002945.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,GCA_000006445.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,GCA_000011425.1,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,0,0,0,0,0,0,0,0,1,0,1,0,0,0,17,0,0
8,GCA_000027005.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,GCA_000091065.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
CSV.write("cazy_df.csv",lj)

"cazy_df.csv"